In [60]:
from methods_fromhome import *
import geopandas as gpd
import pandas as pd

In [61]:
bfe = gpd.read_file('bfe_mre_all.shp')
fsp = gpd.read_file('fsp_sample_mre1.shp')
fl = gpd.read_file('Flowline_Forks.shp')

# Reproject files to UTM
bfe.to_crs(26913, inplace=True)
bfe = bfe[['ELEV', 'geometry']]

fsp.to_crs(26913, inplace=True)
fsp = fsp[['geometry']]
fsp.reset_index(inplace=True)

fl.to_crs(26913, inplace=True)
fl = fl[['geometry']]
fl.reset_index(inplace=True)


In [62]:
# Extend BFEs over FSP Poly
bfe_extend = extend_bfe(bfe, fsp)

# Split FSP Poly by extended BFEs
fsp_s = split_fsp(fsp, bfe_extend)

# Split Flowline by BFEs
fl = split_flowline(fl, bfe)

In [63]:
# 4: Find Forks from Adj flowline segments
def find_forks(flowline):
    forks_list = []
    for i, f in flowline.iterrows():
        p1, p2 = f.geometry.boundary
        p1b = g(p1.buffer(5), 26913)
        p2b = g(p2.buffer(5), 26913)
        p1j = p1b.sjoin(flowline)
        p2j = p2b.sjoin(flowline)
        if p1j.shape[0] == 3:
            forks_list.append(p1j['index_right'].to_list())
        if p2j.shape[0] == 3:
            forks_list.append(p2j['index_right'].to_list())
    
    forks_list.sort()
    forks_list = list(f for f,_ in itertools.groupby(forks_list))
    
    return forks_list


In [64]:
forks_ls = find_forks(fl)

In [65]:
forks_ls

[[0, 11, 10], [18, 6, 20], [22, 19, 21], [25, 17, 24], [28, 26, 27]]

In [66]:
fl['buff'] = fl.apply(lambda x: x.geometry.buffer(1))
fl_buff = fl.set_geometry('buff')
fl_buff = fl_buff[['buff']]
fl = fl[['geometry']]

In [67]:
fl_buff

,buff
0,"POLYGON ((1261601.909 3630922.907, 1261601.965..."
1,"POLYGON ((1261713.187 3633397.150, 1261713.276..."
2,"POLYGON ((1262752.694 3631763.941, 1262752.693..."
3,"POLYGON ((1262743.009 3631705.239, 1262743.882..."
4,"POLYGON ((1262857.369 3631695.911, 1262857.451..."
5,"POLYGON ((1263021.687 3631519.136, 1263021.746..."
6,"POLYGON ((1263125.621 3631385.848, 1263125.716..."
7,"POLYGON ((1261746.265 3631713.811, 1261746.330..."
8,"POLYGON ((1262349.988 3631582.521, 1262350.086..."
9,"POLYGON ((1262352.077 3631582.412, 1262352.184..."


In [68]:
# get fl_segs that are forks
all_fsegs_forks = [segs for e in forks_ls for segs in e]
non_forks = fl.loc[~fl.index.isin(all_fsegs_forks)]
non_forks

,geometry
1,"LINESTRING (1261668.275 3633421.820, 1261712.6..."
2,"LINESTRING (1262749.575 3631811.991, 1262751.6..."
3,"LINESTRING (1262751.519 3631762.830, 1262742.0..."
4,"LINESTRING (1262856.462 3631694.217, 1262857.8..."
5,"LINESTRING (1263011.873 3631530.493, 1263020.8..."
7,"LINESTRING (1261725.140 3631731.195, 1261745.6..."
8,"LINESTRING (1262307.045 3631583.765, 1262349.9..."
9,"LINESTRING (1262349.936 3631581.523, 1262352.0..."
12,"LINESTRING (1262511.616 3631259.587, 1262515.1..."
13,"LINESTRING (1262750.011 3631075.851, 1262756.0..."


In [69]:
fl_non_forks = fl.loc[~fl.index.isin(all_fsegs_forks)].reset_index()
fl_buff_non_forks = fl_buff.loc[~fl_buff.index.isin(all_fsegs_forks)].reset_index()

In [57]:
fl_buff_non_forks = g(fl_buff_non_forks.geometry.to_list(), 26913)
fl_buff_non_forks.to_file('fl_buff_non_forks.shp')

In [71]:
triangles = gpd.GeoDataFrame()
for i, f in fl_non_forks.iterrows():
    fl_buff_i = fl_buff_non_forks.loc[fl_buff_non_forks['index'] == f['index']]
    
    bfe_set = bfe.sjoin(fl_buff_i, how='left', predicate='intersects')
    bfe_set = bfe_set.loc[bfe_set['index_right'].notnull()]
    bfe_set = bfe_set[['ELEV', 'geometry']]
    if bfe_set.shape[0] == 1:
        pass
    else:
        # Getting Z-geom for BFE Points
        bfe_pts = bfe_zpts(bfe_set)

        # BFE Centroid and Flowline interpolation
        fl_i_pts = flowline_interpolation(bfe_set, f, divisions=15, power=2)

        # FSP Simplify
        fsp_pts = fsp_pts_simplify(fsp_s, fl_i_pts, tolerance=3)

        # FSP Interpolation
        fsp_i_pts = IDW(bfe_set, fsp_pts, power=2)

        # Concat and Triangulate
        all_pts = pd.concat([bfe_pts, fl_i_pts, fsp_i_pts], ignore_index=True)
        all_pts_multigeom = MultiPoint(all_pts.geometry.to_list())

        tin = triangulate(all_pts_multigeom)
        tin_df = g(tin, 26913)
        
        # Extract Geom
        final_tin = extract_geom(tin_df)
        triangles = pd.concat([triangles, final_tin], ignore_index=True)



In [83]:
fork_triangles = gpd.GeoDataFrame()
for f in forks_ls:
    print(f)
    f_seg, buff_union = union_fork(f, fl, fl_buff)
    bfe_set = bfe.sjoin(buff_union, how='left', predicate='intersects')
    bfe_set = bfe_set.loc[bfe_set['index_right'].notnull()]
    bfe_set = bfe_set[['ELEV', 'geometry']]

    if bfe_set.shape[0] == 2:
        fake_fork = True
    else:
        fake_fork = False

    f_interp_df = flowline_interpolation(bfe_set, f_seg, 
                                        divisions=15, 
                                        power=2, 
                                        fork='yes', 
                                        non_fork=fake_fork
    )

    # FSP Simplify and interpolation
    fsp_pts = fsp_pts_simplify(fsp_s, f_interp_df, tolerance=3)
    fsp_interp_df = IDW_Forks(bfe_set, 
                                fsp_pts, 
                                power=2, 
                                non_fork=fake_fork
    )

    # Concat and Triangulate
    all_pts = pd.concat([bfe_pts, f_interp_df, fsp_interp_df], ignore_index=True)
    all_pts_multigeom = MultiPoint(all_pts.geometry.to_list())

    tin = triangulate(all_pts_multigeom)
    tin_df = g(tin, 26913)
    
    # Extract Geom
    final_tin = extract_geom(tin_df)
    fork_triangles = pd.concat([fork_triangles, final_tin], ignore_index=True)



[0, 11, 10]
Fake Fork:  [0, 11, 10]


NameError: name 'fake_fork' is not defined

In [80]:
fl.to_file('flowline.shp')

In [81]:
f_seg

,geometry
0,"MULTILINESTRING ((1263798.786 3630640.455, 126..."


In [ ]:






# Run Triangulation Build by Walking FLowlines with 2 BFE intersections
triangles = gpd.GeoDataFrame()
fingers = []
for i, a in adj.iterrows():
    if len(a['index_right']) != 2: 
        pass
    else:
        fl_i = fl.iloc[i]
        fl_buff_i = fl_buff.loc[fl_buff['index'] == i]

        bfe_set = bfe.sjoin(fl_buff_i, how='left', predicate='intersects')
        bfe_set = bfe_set.loc[bfe_set['index_right'].notnull()]
        bfe_set = bfe_set[['ELEV', 'geometry']]

        if bfe_set.shape[0] != 0: 
            # Getting Z-geom for BFE Points
            bfe_pts = bfe_zpts(bfe_set)

            # BFE Centroid and Flowline interpolation
            fl_i_pts = flowline_interpolation(bfe_set, fl_i, divisions=15, power=2)

            # FSP Simplify
            fsp_pts = fsp_pts_simplify(fsp_s, fl_i_pts, tolerance=3)

            # FSP Interpolation
            fsp_i_pts = IDW(bfe_set, fsp_pts, power=2)

            # Concat and Triangulate
            all_pts = pd.concat([bfe_pts, fl_i_pts, fsp_i_pts], ignore_index=True)
            all_pts_multigeom = MultiPoint(all_pts.geometry.to_list())

            tin = triangulate(all_pts_multigeom)
            tin_df = g(tin, 26913)
            
            # Extract Geom
            final_tin = extract_geom(tin_df)
            triangles = pd.concat([triangles, final_tin], ignore_index=True)
            
        else:
            fingers.append(i)
#fl = fl.loc[~fl.index.isin(fingers)]
#fl_buff = fl_buff.loc[~fl_buff.index.isin(fingers)]
        
# fork workflow
if forks_list:
    fork_triangles = gpd.GeoDataFrame()
    for f in forks_list:
        
        f_seg, buff_union = union_fork(f, fl, fl_buff)
        bfe_set = bfe.sjoin(buff_union, how='left', predicate='intersects')
        bfe_set = bfe_set.loc[bfe_set['index_right'].notnull()]
        bfe_set = bfe_set[['ELEV', 'geometry']]

        # Getting Z-geom for BFE Points
        bfe_pts = bfe_zpts(bfe_set)

        # Flowline interpolation
        if bfe_set.shape[0] == 2:
            fake_fork = True
        else:
            fake_fork = False
            
        f_interp_df = flowline_interpolation(bfe_set, f_seg, divisions=15, power=2, fork='yes', non_fork=fake_fork)
            
        print('bfe_set')
        print(bfe_set)
        print('___________')
        print('fork')
        print(f)
        print('____________')
        print(f_interp_df)
        # FSP Simplify and interpolation
        fsp_pts = fsp_pts_simplify(fsp_s, f_interp_df, tolerance=3)
        fsp_interp_df = IDW_Forks(bfe_set, fsp_pts, power=2, non_fork=fake_fork)

        # Concat and Triangulate
        all_pts = pd.concat([bfe_pts, f_interp_df, fsp_interp_df], ignore_index=True)
        all_pts_multigeom = MultiPoint(all_pts.geometry.to_list())

        tin = triangulate(all_pts_multigeom)
        tin_df = g(tin, 26913)
        
        # Extract Geom
        final_tin = extract_geom(tin_df)
        fork_triangles = pd.concat([fork_triangles, final_tin], ignore_index=True)


triangles.to_file('triangles09_4.shp') 
fork_triangles.to_file('fork_triangles09_4.shp')    
"""
print('Triangles')
print(triangles['geometry'].value_counts())
print('_____________')
print('Forks')
print(fork_triangles['geometry'].value_counts())
"""
#all_triangles = pd.concat([triangles, fork_triangles], ignore_index=True)

#all_triangles.to_file(input('Output File: '))